In [5]:
import json, hashlib
from os import urandom, path
from Crypto.PublicKey import RSA 
from Crypto.Cipher import AES
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Util.Padding import pad, unpad
from user import *

In [6]:
def generate_keypair(passwordEncode):                           #2.1   +  2.2
    #Phát sinh cặp khóa (Kpublic, Kprivate) qua RSA
    keyPair = RSA.generate(2048)
    Kpublic = keyPair.publickey().export_key()
    #Băm passphase
    Ksecret = hashlib.sha256(passwordEncode)
    Kprivate = keyPair.export_key()
    
    #Mã hóa Kprivate qua AES
    obj = AES.new(Ksecret.digest(), AES.MODE_CBC)
    encrypted_text = obj.iv + obj.encrypt(pad(Kprivate, AES.block_size))
    
    return Kpublic, Kprivate, encrypted_text

def encryptFile(fileIn, fileOut, userKey_to_send):              #4.2  +  4.3
    bs = AES.block_size #16 bytes
    Ksession = urandom(bs)

    cipher = AES.new(Ksession, AES.MODE_CBC)
    finished = False
    fileOut.write(cipher.iv)
    
    key = RSA.importKey(getKpublic(userKey_to_send))
    publickey = PKCS1_OAEP.new(key)
    
    encrypted = publickey.encrypt(Ksession)                 #4.3

    fileOut.write(encrypted)

    while not finished:
        chunk = fileIn.read(1024 * bs) 
        if len(chunk) == 0 or len(chunk) % bs != 0:   
            padding_length = (bs - len(chunk) % bs) or bs
            chunk += str.encode(padding_length * chr(padding_length))
            finished = True
        fileOut.write(cipher.encrypt(chunk))                    #4.2
    
def decrypt_Kprivate(encrypted_text, passwordEncode):           #5.2
    Ksecret = hashlib.sha256(passwordEncode)
    #print(encrypted_text)
    iv = encrypted_text[:AES.block_size]
    rev_obj = AES.new(Ksecret.digest(), AES.MODE_CBC, iv)
    decrypted_text = rev_obj.decrypt(encrypted_text[AES.block_size:])
    return unpad(decrypted_text,AES.block_size)

def decryptKsession(Kprivate, encrypted_Ksession):              #5.3
    key = RSA.importKey(Kprivate)
    privatekey = PKCS1_OAEP.new(key)
    decrypted = privatekey.decrypt(encrypted_Ksession)
    
    return decrypted

def decryptFile(fileIn, fileOut, userKey_to_send):                     #5.4
    bs = AES.block_size #16 bytes
    iv = fileIn.read(bs)

    encrypted_Kprivate = getEnKprivate(userKey_to_send)
    passwordEncode = getPassword(userKey_to_send).encode()
    cont = fileIn.read(289-33)

    deKprivate = decrypt_Kprivate(encrypted_Kprivate, passwordEncode)
    #deKprivate = deKprivate.replace(b'\n',b'')

    Ksession = decryptKsession(deKprivate, cont) 
    
    cipher = AES.new(Ksession, AES.MODE_CBC, iv)
    next_chunk = ''
    finished = False
    while not finished:
        chunk, next_chunk = next_chunk, cipher.decrypt(fileIn.read(1024 * bs))
        if len(next_chunk) == 0:
            padding_length = chunk[-1]
            chunk = chunk[:-padding_length]
            finished = True 
        fileOut.write(bytes(x for x in chunk)) 

def getUser(user_key):
    users = dict()
    success = False
    if path.exists('data/users_log_data.json'):
        with open('data/users_log_data.json', 'r', encoding='utf-8') as input:
            users  = json.load(input)
    if users:
        for i in users['data']:
            if i['key'] == user_key:
                    user = User(i)
                    success = True
                    return i
    
def getPassword(user_key):
    user = getUser(user_key)
    return  user['info']['password']

def getKpublic(user_key):
    user = getUser(user_key)
    path = 'data/' + user['info']['storage'] + '/Kpublic.pub'
    with open(path, 'rb') as publicFile:
        cont = publicFile.read()
    publicFile.close()
    return cont

def getEnKprivate(user_key):
    user = getUser(user_key)
    path = 'data/' + user['info']['storage'] + '/Kprivate.enc'
    with open(path, 'rb') as privateFile:
        cont = privateFile.read()
    return cont


In [7]:
userKey_to_send = 'm'
passphrase = 'm'
passphrase = passphrase.encode()
pub, pri, pri_en = generate_keypair(passphrase)

In [8]:
with open('infile.docx', 'rb') as in_file, open('outfile.docx', 'wb') as out_file:
    encryptFile(in_file, out_file, userKey_to_send)
in_file.close()
out_file.close()

with open('outfile.docx', 'rb') as in_file, open('outfile_decrypted.docx', 'wb') as out_file:
    decryptFile(in_file, out_file, userKey_to_send)
in_file.close()
out_file.close()